# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 2-2. Эвристика Кернигана—Лина

В этой задаче Вам предлагается добавить к локальному поиска в задаче о сбалансированном разбиении графа эвристику Кернигана—Лина, когда мы, «застряв» в локальном минимуме, тем не менее пытаемся сделать несколько шагов из него, даже если они приводят к временному ухудшению. Надежда здесь на то, что после ухудшения может наступить заметное улучшение результата: нам удастся выпрыгнуть из локального оптимума. Мы рассматриваем безвесовый вариант задачи о разбиении с параметром балансировки $\alpha=\frac{1}{2}$:

**Даны:**
* $G=(V,E)$ — граф без весов на рёбрах

**Найти:**
* Разбиение $V=V'\sqcup V''$, такое, что $V'=\lfloor |V|/2 \rfloor$ и число рёбер между $V'$ и $V''$ минимально возможное.

Сделайте следующее:
* Скачайте файл [`partition-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/partition-instances.zip) и разархивируйте из него файлы со входами задачи.
* Для каждого из графов найдите локальным поиском с эвристикой Кернигана—Лина локально минимальное (по количеству рёбер между частями) разбиение вершин графа на две части, мощности которых отличаются не более чем на единицу. 
* Реализуйте функцию `variable_depth_local_search`; она должна принимать на вход граф в формате, предоставляемом функцией `read_instance`, и возвращать найденное разбиение как множество вершин, лежащих в одной любой из двух компонент разбиения. Ваш локальный поиск должен начинаться с того разбиение, которое уже находится в переменной `starting_point`.
* Подберите для каждого из четырёх входных графов глубину поиска так, чтобы он работал не более 60 секунд на Вашем компьютере, и сохраните информацию о подобранных параметрах и любые свои интересные наблюдения в отдельную ячейку настоящего ipynb-файла.

In [286]:
import random as rn
import time
import math as m
import numpy as np

In [294]:
def read_instance(filename):
    with open(filename, 'r') as file:
        n_vertices = int(file.readline().strip().split()[0])
        vertices, edges = set(range(1, n_vertices + 1)), set()
        for u in range(1, n_vertices + 1):
            for v in map(int, file.readline().strip().split()):
                edges.add((u,v))
        return (vertices, edges)

In [295]:
def add_rm(L:set, a,b):
    L.remove(a)
    L.add(b)


In [296]:
def best_local_partition(graph,p,s,t,flag):
    tmp = 0 
       # t = set([])
    #будем рассматривать разницу расстояний между вершинами
    Distance = 0
    ind = 0
    n = len(graph[1])
    Dist = [0]*n
    for a,b in graph[1]:  #рассмотрим пары на принадлежность partitions
            if ((a in p) == (b in p)):
                if flag:
                    print('\nChecking graph...\n')
                    flag = False
                Dist[a-1] += 1
                Dist[b-1] += 1
            else:
                Dist[a-1] -=1
                Dist[a-1] -=1
    tmp_p = p.copy()
    minDist = 50000
    bestV0, bestV1 = 0, 0
    L = tmp_p - s
    R = graph[0] - p 
    R-= t
    for a in L: 
        for b in R:
            ind = 0
            e1 = (a,b)
            e2 = (b,a)
            if e1 in graph[1] or e2 in graph[1]: 
                ind += 1
            NewDist = Dist[a - 1] + Dist[b - 1] + 2*ind
            if NewDist < minDist:
                minDist = NewDist
                bestV0, bestV1 = a,b
    L1 = L.copy()
    R1 = R.copy()
    add_rm(L1,bestV0,bestV1)
    add_rm(R1,bestV1,bestV0)
    return (L1,R1)
    
                

In [297]:
def variable_depth_local_search(graph,depth):
    #depth = [1,2,3,4]
    flag = 1
    s = set([])
    t = set([])
    starting_point = set(range(1, len(graph[0]) // 2 + 1))
    # my code here
    depth_level = 0 #глубина
    V,E = graph
    n = len(V)
    L = starting_point 
    L1 = L.copy()
    R = V - starting_point
    tmp1 = 0
    for e in E:
        a,b = e
        if (a in L and b in R)or(a in R and b in L): #если лежат в разных долях
            tmp1 += 1
        while depth > depth_level:
            BestL, BestR = best_local_partition(graph, starting_point,s,t,flag)
            flag = 0
            tmp = 0
            for w in E:
                u,v = w
                if (u in L and v in R)or(v in L and u in R):
                    tmp+=1
            if tmp > tmp1:
                depth_level += 1
            else:
                tmp1 = tmp
                depth = 0
                L1 = BestL
            L = BestL
            R = BestR
            
    return BestL

In [306]:
%%time
def get_quality(graph, partition_part):
    if not (partition_part <= graph[0]) or abs(len(partition_part) - len(graph[0]) / 2) > 0.6:
        return -1
    other_part = set(graph[0]) - partition_part
    return sum(1 for edge in graph[1] if set(edge) <= partition_part or set(edge) <= other_part )

def run_all():
    depth = [1,1,1,1]
    filenames = ['add20.graph', 'cti.graph', 't60k.graph', 'm14b.graph']
    for filename, depth in zip(filenames, depth):
        instance = read_instance(filename)
        print('Solving instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        quality = get_quality(instance, variable_depth_local_search(instance, depth))
        time_elapsed = time.monotonic()-time_start
        print(' done in {:.2} seconds with quality {}'.format(time_elapsed, quality))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 10.3 µs


In [ ]:
%%time
run_all()

Solving instance add20.graph…
Checking graph...

 done in 0.73 seconds with quality 11084
Solving instance cti.graph…
Checking graph...

 done in 4.8e+01 seconds with quality 94056
Solving instance t60k.graph…
Checking graph...

 done in 6.8e+02 seconds with quality 178240
Solving instance m14b.graph…
Checking graph...



## Выводы
(Здесь опишите свои наблюдения и подобранные параметры для каждогр из четырёх входных графов.)

In [ ]:
Сначала рассмотрим случайные параметры (3 запуска)
Первый пример был решен очень быстро -- можно сказать, что для него бы подошли любые разумные параметры. 
Второй граф был решён примерно за полминуты, что тоже достойно.
Последний отработал на глубине 3 15 минут и 
не загрузился. Видим, что с глубиной растет время работы.
Попробуем запустить алгоритм для единичной глубины и посмотрим на уровень качества. Подобным поиском приходим к выводу
что нам подходит последовательность [1,1,1,1] уровней глубины и с достаточно хорошим качеством.
К слову, уровень качества не очень сильно изменится при значительном увеличении глубины, 
а вот время работы возрастет сильно.